# <center> Replicate 3 SNV and gene trajectories in Lactobacillus evolution experiment </center>

When we last talked, we discussed plotting allele frequency trajectories for Replicate 3 specifically. As I understood it, Karina's analysis indicated that there may have been a SNV replacement in Replicate 3. Producing these plots was meant to head off possible reviewer critiques. I have produced these plots, as well as additional plots showing the dynamics of gene "copy number", a proxy for the frequency of a gene in the population. 

First, I chose the hybrid assembly of the ancestral strain (LpWF-Nanopore) as a reference genome against which to align reads. As we only expect to see gene (particularly plasmid) loss during the course of the experiment, the gene content of this ancestral strain should contain the gene contents of all derived strains. The hybrid assembly should contain both core and accessory genome elements. 

Next, I mapped reads to this reference genome with MIDAS, and called SNVs and gene copy numbers in each sample. In MIDAS, the gene "copy number" is the mean read coverage of that gene relative to the mean read coverage of 13 single-copy, universal genes. In practice, genes present in all cells in a population will have copy number ~1, while a gene present in only 10% of cells will have copy number 0.1. 

In [ ]:
from scipy.optimize import curve_fit

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fit_utils import *

import matplotlib


#matplotlib.rc('text', usetex=True)
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
#matplotlib.verbose.level = 'debug-annoying'

matplotlib.rc('axes', linewidth=.5)
matplotlib.rc('font', weight='bold')

label_size=15
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size 


#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
#matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

import seaborn as sns

color_mood = ["#a63a33","#f1c132","#3c414d","#d7d7de","#aaaaaa"]

font = {'family': 'sans-serif',
        'color':  'k',
        'size': 40,
        "weight":"bold"
        }

plt.style.use('bmh')


In [ ]:
base_snps_dir = "/u/scratch/r/rwolff/Evolution_Experiment/midas_output/merged_midas_output/snps/Lactobacillus"

c3 = ['WF_parent','Rep3_Pass5','Rep3_Pass9','Rep3_Pass16','Rep3_Pass23','Rep3_Pass30','Rep3_Pass37','Rep3_Pass44','Rep3_Pass51']


In [ ]:
gene_descriptions = pd.read_csv("genes_descriptions.csv",index_col=0,sep="\t")
snps_info = pd.read_csv(f"{base_snps_dir}/snps_info.txt",sep="\t",index_col=0)

## <center> SNV frequencies </center> 

Across all time points in Replicate 3, I detected a total of 721 SNVs—defined as any site which was variable in any sample and had sufficient coverage. Specifically, I used the following criteria:

* **min_depth**: minimum coverage at a site to be considered (set to 5 in the plots below)
* **min_count**: minimum number of samples a site must have coverage $\geq$ min_depth (set to 7)

These thresholds were chosen to include only sites which are consistently detected to be polymorphic, and are unlikely to result from one-off misalignments.



In [ ]:
min_depth = 5
min_count = 7

df = pd.read_csv(f"{base_snps_dir}/snps_freq.txt",sep="\t",index_col=0)


In [ ]:
for s in snps_info.index:
    if snps_info.loc[s,"ref_allele"] == snps_info.loc[s,"major_allele"]:
        pass
    else:
        df.loc[s] = 1 - df.loc[s]

In [ ]:
depths = pd.read_csv(f"{base_snps_dir}/snps_depth.txt",sep="\t",index_col=0)
df_freq = df.where(depths > min_depth,np.nan)
df_freq_covered = df_freq[c3].loc[df_freq[c3].T.count() > min_count]

Here is a figure showing the raw SNV trajectories 

In [ ]:
c = c3

fig,ax = plt.subplots(figsize=(16,8))
ax.set_facecolor("white")
ax.grid(False)

fig.suptitle("Replicate 3 SNV frequencies",size=25)

ax.plot(df_freq_covered.T.values,color="grey",alpha=.6)

ax.set_ylim([-0.025,1.025])
ax.set_xticks(range(len(c)))
ax.set_xticklabels([cc.split("_")[1] for cc in c]);
ax.set_xlabel("Passage",size=25)
ax.set_ylabel("Allele frequency",size=25);

Visually inspecting these raw trajectories, we see that 3 SNVs appear to sweep. If we color these sweeping SNVs, we can more easily see these trajectories:

In [ ]:
snv1 = 3193223
snv2 = 3373442
snv3 = 3373497

In [ ]:
fig,ax = plt.subplots(figsize=(16,8))
ax.set_facecolor("white")
ax.grid(False)
fig.suptitle("Replicate 3 SNV frequencies",size=25)

ax.plot(df_freq_covered.drop(df_freq_covered[c]["WF_parent"].idxmax()).T.values,color="grey",alpha=.1)

ax.plot(df_freq_covered.loc[snv1],color=color_mood[0],zorder=10,lw=7.5)
ax.plot(df_freq_covered.loc[[snv2,snv3]].T,color=color_mood[1],zorder=10,lw=7.5)

ax.set_ylim([-0.025,1.025])
ax.set_xticks(range(len(c)))
ax.set_xticklabels([cc.split("_")[1] for cc in c]);
ax.set_xlabel("Passage",size=25)
ax.set_ylabel("Allele frequency",size=25);

Using the gene  contained in the .gff file Karina sent me, we can inspect the functional annotation of these sites, to see if they might be putative targets of selection.

### <font color='#a63a33'>SNV 1</font>

The first SNV to sweep, shown in  <font color='#a63a33'>red</font>, is at a 1D site located in a "Histidine kinase of the competence regulon ComD" gene

In [ ]:
snps_info.loc[snv1]

In [ ]:
gene_descriptions.loc[snps_info.loc[snv1].gene_id]

### <font color='#f1c132'>SNVs 2 & 3</font>

The second pair of SNVs to sweep are shown in <font color='#f1c132'>yellow</font>. These SNVs are 55 bp away from one another in a "Mobile element protein" gene. One (site_id: 3373442) is a 4D site, while the other (site_id: 3373497) is a 1D site. The mutation at the 1D site flips the codon from a stop codon to a glutamine.

In [ ]:
snps_info.loc[[snv2,snv3]]

In [ ]:
gene_descriptions.loc[snps_info.loc[snv3].gene_id]

Two out of these three sites are non-synonymous variants, while the third (synonymous) site is obviously linked to the second. All three sites had at least low levels of polymorphism in the ancestral population. Interestingly, both genes might putatively be involved in/affected by horizontal gene transfer/recombination. The mobile element protein is only 191 base pairs long, and may very well be a TE, while the histidine kinase is situated in a pathway involved in genetic transformation. There seems to be a fairly large body of research on this *com* operon (e.g. [Ianelli 2005](https://academic.oup.com/femsle/article/252/2/321/530087)). 

Given what I understand of the experiment, I find it not unreasonable to suppose that these genes might harbor loci which are under positive selection. 

### <center> Single-locus dynamics </center>

It is evident that all three SNVs begin at low frequency, and approach fixation by the end of the experiment, but are there dynamics consistent with positive selection?

The simplest model of single-locus evolutionary dynamics predicts that a site experiencing positive selection of strength $s$ will be described by the following ODE:

\begin{equation}
    \frac{df}{dt} = sf(1-f)
\end{equation}

where the units of time are, for the moment, unspecified.

For a given initial frequency $p_0$, this ODE is easily solved:

\begin{equation}
    f(t) = \frac{p_0e^{st}}{1+p_0(e^{st} - 1)}
\end{equation}

Assuming that the strength of selection per unit time is constant, and ignoring stochasticity due to drift during growth and sampling noise induced by serial passaging, this equation (known as a sigmoid curve) will describe the time evolution of a single locus. Since we know the initial frequency of each allele in the ancestral population, we can easily fit a sigmoid curve for any allele frequency trajectory we desire by fitting the selection parameter $s$. Let's try it for the red SNV (SNV 1):

In [ ]:
class fitClass:

    def __init__(self):
        pass

    def sigmoid(self, t, s):
        return self.p0*np.exp(s*t)/(1 + self.p0*(np.exp(s*t) - 1))
    
    def delay_sigmoid(self,t,s,t_s):
        
        return self.p0*(t<=t_s) + self.p0*np.exp(s*(t - t_s))/(1 + self.p0*(np.exp(s*(t - t_s)) - 1))*(t>t_s)

In [ ]:
xdata = [0,5,9,16,23,30,37,44,51]
ydata_snv1 = df_freq_covered.loc[snv1].values

inst_snv1 = fitClass()

inst_snv1.p0 = ydata_snv1[0]
coeffs, coeffs_cov = curve_fit(inst_snv1.sigmoid, xdata, ydata_snv1)
s_snv1 = coeffs[0]

t_range = np.linspace(0,51,10000)
sig_pred = inst_snv1.sigmoid(t_range,s_snv1)

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

fit_eq = r"$f(t) = \frac{p_0e^{{s}t}}{1+p_0(e^{st} - 1)}$" 
l2 = fr"$p_0 = {np.around(inst_snv1.p0,2)}$"
l3 = fr"$s = {np.around(s_snv1,2)}$"
label1 = "\n" + fit_eq + "\n" + l2 + ", " + l3 

ax.set_facecolor("white")

ax.plot(xdata,ydata_snv1,color=color_mood[0],lw=5,label="SNV 1 frequency")
ax.scatter(xdata,ydata_snv1,facecolor=color_mood[0],edgecolor="k",s=75,zorder=10)

ax.plot(t_range,sig_pred,color="k",ls="--",lw=5,label=label1)

ax.set_xlabel("Time (days)",size=25)
ax.set_ylabel("Allele frequency",size=25)

fig.legend(bbox_to_anchor=(.92, .88), loc='upper left', borderaxespad=0,prop={'size': 20});



The fit of the sigmoid curve is excellent. The dynamics at this site are almost as exactly as predicted under positive selection by our single-locus model, with a selective advantage of $s=0.31$/day. Assuming 10 generations per day in media, this translates to a roughly $3.8\%$ fitness advantage for the SNV 1 mutants over the wild-type per generation.

Now, let's try the same analysis on SNV 3:

In [ ]:
ydata_snv3 = df_freq_covered.loc[snv3].values

inst_snv3 = fitClass()

inst_snv3.p0 = ydata_snv3[0]
coeffs, coeffs_cov = curve_fit(inst_snv3.sigmoid, xdata, ydata_snv3)
s_snv3 = coeffs[0]

t_range = np.linspace(0,51,10000)
sig_pred_snv3 = inst_snv3.sigmoid(t_range,s_snv3)


In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

fit_eq = r"$f(t) = \frac{p_0e^{{s}t}}{1+p_0(e^{st} - 1)}$" 
l2 = fr"$p_0 = {np.around(inst_snv3.p0,2)}$"
l3 = fr"$s = {np.around(s_snv3,2)}$"
label2 = "\n" + fit_eq + "\n" + l2 + ", " + l3 
ax.set_facecolor("white")

ax.plot(xdata,ydata_snv3,color=color_mood[1],lw=5,label="SNV 3 frequency")
ax.scatter(xdata,ydata_snv3,facecolor=color_mood[1],edgecolor="k",s=75,zorder=10)

ax.plot(t_range,sig_pred_snv3,color="k",ls="--",lw=5,label=label2)

ax.set_xlabel("Time (days)",size=25)
ax.set_ylabel("Allele frequency",size=25)

fig.legend(bbox_to_anchor=(.92, .88), loc='upper left', borderaxespad=0,prop={'size': 20});



Evidently, this fit is quite poor. It seems that the sigmoid curve initially *overestimates* the frequency trajectory, and then later *underestimates* it. Rather than immediately rising in frequency, as predicted by the model, it appears that the allele's trajectory can be split into two phases. In the first phase (roughly timepoints 0-25), the allele remains near its initial value. Then, in the second phase (timepoints 25-51), the allele begins to increase rapidly in frequency and quickly fixes. These kind of dynamics might be expected if the allele was initially neutral, but at some point became advantageous. 

So, what if the fitness advantage only switches on at some point during the experiment? We can model this situation by supposing that the allele is initially neutral ($s_{\text{initial}}=0$), but subsequently becomes advantageous ($s > 0$). These dynamics can be described with the modified equation:

\begin{equation}
    \begin{split}
        \frac{df}{dt} &= s_{\text{initial}}f(1-f)\left(1 - H(t_s - t)\right) + sf(1-f)H(t_s - t)  \\
        &= sf(1-f)H(t_s - t) 
    \end{split}   
\end{equation}

where $H(t)$ is the [Heaviside function](https://en.wikipedia.org/wiki/Heaviside_step_function) (complicated expression for a simple concept: $H(t)$ is equal to 0 when $t<0$ and 1 when $t>0$. Here, we use it to "turn on" selection after $t_s$.)

Again, this ODE is easily solved, and results in a piecewise function:

\begin{equation}
f(t)= 
    \begin{cases}
        p_0 & \quad \text{if } t \leq t_s \\
        \frac{p_0e^{s(t - t_s)}}{1+p_0(e^{s(t - t_s)} - 1)} & \quad \text{if } t > t_s
    \end{cases} 
\end{equation}

Now, let's try fitting this new "delayed" sigmoid curve to our SNV 3 trajectory:

In [ ]:
ydata_snv3 = df_freq_covered.loc[snv3].values

inst_snv3 = fitClass()

inst_snv3.p0 = ydata_snv3[0]
## provide an initialization here because otherwise there are convergence issues
coeffs, coeffs_cov = curve_fit(f=inst_snv3.delay_sigmoid, xdata=xdata, ydata=ydata_snv3,p0=np.array([0.1,15]))
s_snv3 = coeffs[0]
t_s_snv3 = coeffs[1]

t_range = np.linspace(0,51,10000)
sig_del_pred_snv3 = inst_snv3.delay_sigmoid(t_range,s_snv3,t_s_snv3)


In [ ]:
fig,ax = plt.subplots(figsize=(12,8))

fit_eq = r"$f(t)= \begin{cases} p_0 & \quad \text{if } t \leq t_s \\ \frac{p_0e^{s(t - t_s)}}{1+p_0(e^{s(t - t_s)} - 1)} & \quad \text{if } t > t_s\end{cases}$"
l2 = fr"$p_0 = {np.around(inst_snv3.p0,2)}$"
l3 = fr"$t_s = {np.around(t_s_snv3,2)}, s = {np.around(s_snv3,2)}$"

label3 = fit_eq + "\n" + l2 + "," + l3

ax.set_facecolor("white")

ax.plot(xdata,ydata_snv3,color=color_mood[1],lw=5,label="SNV 3 frequency")
ax.scatter(xdata,ydata_snv3,facecolor=color_mood[1],edgecolor="k",s=75,zorder=10)

ax.plot(t_range,sig_del_pred_snv3,color="k",ls="--",lw=5,label=label3)

ax.set_xlabel("Time (days)",size=25)
ax.set_ylabel("Allele frequency",size=25)

ax.axvline(t_s_snv3,lw=1.5,color="blue")

ax.annotate(r'$t_s$', xy=(t_s_snv3-2.5, .85),size=40,color="blue")

fig.legend(bbox_to_anchor=(.92, .88), loc='upper left', borderaxespad=0,prop={'size': 20});




Now, incorporating the delay, we obtain an excellent fit of the data. Before $t_s$, the frequency of the allele remains near $p_0$, with two timepoints slightly above, and two timepoints slightly below. After $t_s$, we see rapid sigmoidal increase in frequency, with the allele nearly fixing by the end of the experiment. During the sigmoidal phase, we observe a selection coefficient of $s = 0.42$/day, or roughly a $5\%$ fitness advantage per generation over the wild type.

If we plot the two sweep events on top of one another, we see that SNV 3 appears to begin its sweep just as SNV 1 nears fixation:

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
ax.set_facecolor("white")

fit_eq = r"$f(t) = \frac{p_0e^{{s}t}}{1+p_0(e^{st} - 1)}$" 
l2 = fr"$p_0 = {np.around(inst_snv1.p0,2)}$"
l3 = fr"$s = {np.around(s_snv1,2)}$"
label1 = "\n" + fit_eq + "\n" + l2 + ", " + l3 + "\n"

ax.plot(xdata,ydata_snv1,color=color_mood[0],lw=5,label="SNV 1 frequency")
ax.scatter(xdata,ydata_snv1,facecolor=color_mood[0],edgecolor="k",s=75,zorder=10)

ax.plot(t_range,sig_pred,color="k",ls="--",lw=5,label=label1)

fit_eq = r"$f(t)= \begin{cases} p_0 & \quad \text{if } t \leq t_s \\ \frac{p_0e^{s(t - t_s)}}{1+p_0(e^{s(t - t_s)} - 1)} & \quad \text{if } t > t_s\end{cases}$"
l2 = fr"$p_0 = {np.around(inst_snv3.p0,2)}$"
l3 = fr"$t_s = {np.around(t_s_snv3,2)}, s = {np.around(s_snv3,2)}$"

label3 = "\n" + fit_eq + "\n" + l2 + ", " + l3

ax.set_facecolor("white")

ax.plot(xdata,ydata_snv3,color=color_mood[1],lw=5,label="SNV 3 frequency")
ax.scatter(xdata,ydata_snv3,facecolor=color_mood[1],edgecolor="k",s=75,zorder=10)

ax.plot(t_range,sig_del_pred_snv3,color=color_mood[4],ls="--",lw=5,label=label3)

ax.set_xlabel("Time (days)",size=25)
ax.set_ylabel("Allele frequency",size=25)

ax.axvline(t_s_snv3,lw=1.5,color="blue")
ax.annotate(r'$t_s$', xy=(t_s_snv3-2.5, .65),size=40,color="blue")

fig.legend(bbox_to_anchor=(.92, .88), loc='upper left', borderaxespad=0,prop={'size': 20});



On the basis of this figure, we might be led to believe that the population as a whole has evolved by successive substitutions—first with the sweep of SNV 1, and then with the sweep of SNVs 2 & 3 (which are undoubtedly linked). However, this simple picture is not correct, as we shall see below. 

## <center> Gene copy number dynamics </center> 

While somewhat beyond the purview of the original assignment, I wanted to inspect the copy number dynamics of the genes in Replicate 3. Particularly, I wanted to see if the genes in which SNVs 1, 2, & 3 lie exhibited discussed any interesting/unusual dynamics. 

In the figure below, I plot the copy number dynamics of all genes. Again, MIDAS copy number is a proxy for the frequency of a gene in a population. I maintain the same color scheme as above—the competence regulon histidine kinase gene (where SNV 1 is located) is colored <font color='#a63a33'>red</font>, while the mobile element protein gene (where SNVs 2 & 3 are) is colored <font color='#f1c132'>yellow</font>. Additionally, there was a cluster of genes which clearly were lost during the experiment, and I have colored these <font color='green'>green</font>—presumably, a large percentage of these are colonization island genes. 

In [ ]:
base_genes_dir = "/u/scratch/r/rwolff/Evolution_Experiment/midas_output/merged_midas_output/genes/Lactobacillus"

In [ ]:
dfg = pd.read_csv(f"{base_genes_dir}/genes_copynum.txt",sep="\t",index_col=0)
dfg.index = [d.split("|")[1] for d in dfg.index]
dfg = dfg[c]

## by visual inspection, 10% appears to be a good cutoff for the cluster of lost genes
lost_genes = dfg.loc[dfg["Rep3_Pass51"] < 1e-1]

In [ ]:
dfc = pd.read_csv(f"{base_genes_dir}/genes_reads.txt",sep="\t",index_col=0)
dfc.index = [d.split("|")[1] for d in dfc.index]
dfc = dfc[c]


In [ ]:
fig,ax = plt.subplots(figsize=(16,8))
ax.set_facecolor("white")
ax.grid(False)
fig.suptitle("Replicate 3 gene copy number",size=25)

ax.plot(xdata,dfg.T,color="grey",alpha=.1)

ax.plot(xdata,lost_genes.T.values,color="green",alpha=1)

ax.plot(xdata,dfg.loc[snps_info.loc[snv1].gene_id],color=color_mood[0],alpha=1,lw=7.5)
ax.plot(xdata,dfg.loc[snps_info.loc[snv3].gene_id],color=color_mood[1],alpha=1,lw=7.5)

ax.set_xlabel("Time (days)",size=25)
ax.set_ylabel("Gene copy number",size=25)
ax.semilogy();



The histidine kinase maintains relative copy number ~1 throughout the experiment (as do the bulk of genes in the main cloud centered around 1—some stochastic variation in copy number due to read sampling noise is expeceted). Based on this graph, it is unlikely that any appreciable fraction of the population lost this gene. 

However, the mobile element protein has a different, and very interesting trajectory. Initially, it seems to lie within the cluster of green genes which are on their way to being lost. However, in the middle of the timecourse, the yellow gene beings to diverge from this cluster. It increases marginally in copy number (i.e. frequency in the population), and ultimately ends up with a copy number around 0.25. 

Comparing the SNV and gene trajectories yields interesting information. Particularly, in the figure below, we see that the yellow mobile element protein begins to separate from the green cluster at around the same timepoint ($t_s$) as the sweep of SNVs 2 & 3. Is this a coincidence, or could the SNV change be implicated in this gene being saved from extinction? 


In [ ]:
fig,axs = plt.subplots(2,1,figsize=(16,12))
fig.tight_layout()
fig.subplots_adjust(top=0.9)

axs = axs.ravel()

axs[0].set_facecolor("white")
axs[0].grid(False)
axs[1].set_facecolor("white")
axs[1].grid(False)

fig.suptitle("Replicate 3 combined",size=25)

axs[0].plot(xdata,ydata_snv1,color=color_mood[0],lw=5,label="SNV 1 frequency")
axs[0].scatter(xdata,ydata_snv1,facecolor=color_mood[0],edgecolor="k",s=75,zorder=10)

axs[0].plot(t_range,sig_pred,color="k",ls="--",lw=5,label=label1)

axs[0].plot(xdata,ydata_snv3,color=color_mood[1],lw=5,label="SNV 3 frequency")
axs[0].scatter(xdata,ydata_snv3,facecolor=color_mood[1],edgecolor="k",s=75,zorder=10)

axs[0].plot(t_range,sig_del_pred_snv3,color=color_mood[4],ls="--",lw=5,label=label3)

axs[0].set_ylabel("Allele frequency",size=25)

axs[0].axvline(t_s_snv3,lw=1.5,color="blue")
axs[0].annotate(r'$t_s$', xy=(t_s_snv3-2.5, .65),size=40,color="blue")

axs[0].set_xticklabels([]);

axs[1].plot(xdata,dfg.T,color="grey",alpha=.1)

axs[1].plot(xdata,lost_genes.T.values,color="green",alpha=1)

axs[1].plot(xdata,dfg.loc[snps_info.loc[snv1].gene_id],color=color_mood[0],alpha=1,lw=7.5)
axs[1].plot(xdata,dfg.loc[snps_info.loc[snv3].gene_id],color=color_mood[1],alpha=1,lw=7.5)

axs[1].axvline(t_s_snv3,lw=1.5,color="blue")
axs[1].annotate(r'$t_s$', xy=(t_s_snv3-2.5, 1e-3),size=40,color="blue")

axs[1].set_xlabel("Time (days)",size=25)
axs[1].set_ylabel("Gene copy number",size=25)
axs[1].semilogy();



In either case, it would seem that SNVs 2 & 3 arise and fix within a subpopulation of cells which bear this gene, rather than sweeping through the population as a whole, as SNV 1 appears to do. 

### <center> Gene losses </center>

Lastly, I inspected the annotations of the green colored genes which were lost over the course of the sampling period. I counted up the categories of these genes:


In [ ]:
lost_gene_descs_counts = gene_descriptions.loc[lost_genes.index].groupby("Description").size()
lost_gene_descs_counts.sort_values(ascending=False)

#all_gene_descs_counts = gene_descriptions.loc[dfg_normed.index].groupby("Description").size()

There are striking patterns in the categories of gene lost. Firstly, all 14 genes with the description "Predicted cell-wall-anchored protein SasA (LPXTG motif)" which were present in the ancestral population are lost. The next most common category of lost genes are "Mobile element proteins." Interestingly, a large number of the remaining lost genes are either phage (e.g. "phage major spike protein") or plasmid (e.g. "putative plasmid partition protein") -related genes.

Of the 49 genes lost, 43 are located on a single contig (4itera_trimm_contig_2), while the other 6 are located on a different contig (4itera_trimm_contig_7). 4itera_trimm_contig_7, however, was only made up of 6 genes initially, so the entire contig was lost during the experiment. Presumably, this was a colonization island. Additionally, all 6 of these genes have descriptions which indicate that they are phage related genes. 4itera_trimm_contig_2 originally had ~200 genes, and so approximately 1/5 of the genes on this contig were lost. The mobile element protein gene on which the yellow SNVs are located is also located on 4itera_trimm_contig_2. 

These gene losses are likely well known to you, but this is figure does quite nicely capture the underlying evolutionary dynamics of the experiment. 

In [ ]:
# ann_dir = "/u/scratch/r/rwolff/Evolution_Experiment/Annotation"
# midas_dir = "/u/scratch/r/rwolff/Evolution_Experiment/midas_db/Lactobacillus"
# gff = "LpWF-Nanopore"

# df = pd.read_csv(f"{ann_dir}/{gff}.gff",skiprows=1,sep="\t",header=None)

# columns = ["gene_id","scaffold_id","start","end","strand","gene_type"]

# df = df[[0,2,3,4,6,8]]

# df.columns = ["scaffold_id","gene_type","start","end","strand","gene_id"]

# df = df[columns]

# df["gene_id"] = [g.split("|")[1].split(";")[0] for g in df["gene_id"]]

# df.index = df.gene_id

# df.loc[lost_genes.index].groupby("scaffold_id").size()

# df.groupby("scaffold_id").size()
# df.loc[lost_genes.index].scaffold_id == "4itera_trimm_contig_7"

## <center> Concluding thoughts </center>

By inspecting the SNV and gene frequency trajectories, I don't think it's at all clear that the SNV changes are purely passengers. To summarize:

* 2 of the 3 SNV changes are non-synonymous, and the third is clearly linked to the second. Additionally, thee second affects a stop codon, and so might be expected to have a large effect.
* All 3 SNVs are located in genes which seem like possible targets for selection. SNV 1, in particular, may affect the efficacy of genetic transformation.
* All 3 SNVs exhibit sigmoid dynamics, exactly as expected were they truly under selection.

I think that it's entirely possible that these single nucleotide evolutionary dynamics are not independent of the gene loss dynamics (i.e. loss of colonization island). Rather, there may be epistatic effects/feedbacks between these SNVs and the genes. For instance, SNV 1 might in theory render cells non-competent for transformation, hastening the loss of the colonization island by preventing its re-uptake from the environment. Similarly, the evolutionary changes at SNVs 2 and 3 may prevent that mobile element protein from being entirely lost from the population.